In [1]:
import pandas as pd
import matplotlib.pyplot
import numpy as np

class ivSignal(object): 
    
    def __init__(self, i_series, v_series, fs, fs_net=50):
        self.v_series = v_series
        self.i_series = i_series
        self.fs = fs
        self.fs_net = fs_net
    
    def subsample(self, dec_factor = 2):
        pass
    
    def calculateIV(self, window=None):
        if window == None:
            window = self.fs/self.fs_net
            
        self.V_series = self.v_series
        self.V_series = self.V_series.apply(np.square)
        self.V_series = self.V_series.rolling(window=window, center=True).mean()
        self.V_series = self.V_series.apply(np.sqrt)
        return None
    
    def calculateFFT(self, start=0, periods=1):
        samples_per_period = self.fs/self.fs_net
        window = samples_per_period*periods
        n = next_pow2(window)
        start_i = self._find_zerocrossing(self.v_series[start:])
        df = pd.DataFrame()
        if not (start_i == None):
            v_fourier = 1j*np.fft.fft(self.v_series[start_i:start_i+window], n)/n
            i_fourier = 1j*np.fft.fft(self.i_series[start_i:start_i+window], n)/n
            v_fourier = np.fft.fftshift(v_fourier)
            i_fourier = np.fft.fftshift(i_fourier)
            f = np.arange(n)
            f = (f-float(n)/2)*self.fs/float(n)
            df = pd.DataFrame({'Frequency (Hz)': f, 'Current (A)': i_fourier, 'Voltage (V)' : v_fourier})
            df = df.set_index('Frequency (Hz)')
        return df
    
    def calculatePQ(self):
        pass
    
    def calculatePhi(self):
        samples_per_cycle = self.fs/self.fs_net
        phi = list()
        for i in range(0, self.v_series.size, samples_per_cycle):
            print(i)
            df = self.calculateFFT(start=i, periods=8)
            v_fund = df.loc[self.fs_net]['Voltage (V)']
            i_fund = df.loc[self.fs_net]['Current (A)']
            power_fund = v_fund*i_fund
            phi.append(np.arctan(np.imag(power_fund)/np.real(power_fund)))
        return pd.Series(phi)
    
    def _find_zerocrossing(self, series):
        for (i, y) in series.iteritems():
            if i < series.size:
                if not (np.sign(series.iloc[i]) <> np.sign(series.iloc[i+1])):
                    return i
        return None
    
    
def next_pow2(n):
    return int(np.square(np.ceil(np.sqrt(n))))
        
        